## Investigating Californian Housing Prices with Regression Models 

In [ ]:
# Imports

from sklearn.linear_model import LinearRegression 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.datasets import fetch_california_housing
import pandas as pd


import folium
from folium.plugins import HeatMap

from sklearn.linear_model import LinearRegression 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt
import numpy as np




In [ ]:
# Fetch the California housing dataset
data = fetch_california_housing(as_frame=True)

# Convert to a pandas DataFrame
housing_data = data.frame

# Display the first few rows
print(housing_data.head())

# Save locally if needed
housing_data.to_csv("housing_data.csv", index=False)

# Basic info about the dataset
print(housing_data.info())

In [ ]:
print("-- INFO--")
print(housing_data.info())

print("-- DESCRIBE--")
print(housing_data.describe())

housing_data

## 1.2 Visulizing Price Using Maps 

In [ ]:

# Create a base map centered on California
california_map = folium.Map(location=[housing_data["Latitude"].mean(), housing_data["Longitude"].mean()], 
                            zoom_start=6)

# Add a heat map layer for MedHouseVal
heat_data = [
    [row['Latitude'], row['Longitude'], row['MedHouseVal']] 
    for _, row in housing_data.iterrows()
]

HeatMap(heat_data, radius=10).add_to(california_map)

# Save and display the map
california_map.save("california_housing_map.html")
california_map

### Correlation HeatMap
Looks like the Median Income is largely corrolated with the price of the house. This makes sense as people with higher income also tends to afford the most expensive houses.

In [ ]:
# Calculate the correlation matrix
import matplotlib.pyplot as plt 
import seaborn as sns

# Correlation Variables. 
correlation_matrix = housing_data.corr()

# Plot the correlation matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True, cbar=True)
plt.title("Correlation Matrix of Housing Data")
plt.tight_layout()
plt.show()



In [ ]:

X = housing_data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]
y = housing_data['MedHouseVal']

model =  LinearRegression()
model.fit(X, y)

# Model summary
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)
predictions = model.predict(X_test)

print("R² Score:", r2_score(y_test, predictions))
print("Mean Squared Error:", mean_squared_error(y_test, predictions))

In [ ]:

# Scatter plot of Actual vs. Predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, predictions, alpha=0.6, color='blue', label='Predicted vs Actual')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2, label='Perfect Fit')
plt.title("Actual vs Predicted Housing Prices")
plt.xlabel("Actual Median House Value")
plt.ylabel("Predicted Median House Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

from scipy.stats import shapiro

# Residuals calculation

residuals = y_test - predictions

# Residuals scatter plot
plt.figure(figsize=(12, 6))

# Subplot 1: Residuals vs Predicted Values
plt.subplot(1, 2, 1)
plt.scatter(predictions, residuals, alpha=0.6, color='purple', label='Residuals')
plt.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Zero Residual Line')
plt.title("Residuals vs Predicted Values")
plt.xlabel("Predicted Median House Value")
plt.ylabel("Residuals")
plt.legend()
plt.grid(True)

# Subplot 2: Histogram and KDE of Residuals
plt.subplot(1, 2, 2)
sns.histplot(residuals, kde=True, bins=100, color='blue', label='Residuals Distribution')
plt.axvline(residuals.mean(), color='red', linestyle='--', linewidth=2, label='Mean')
plt.title("Histogram and KDE of Residuals")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Shapiro-Wilk test for normality
stat, p = shapiro(residuals)
if p > 0.05:
    print(f"Shapiro-Wilk Test: p-value = {p:.10f} (Residuals are normally distributed)")
else:
    print(f"Shapiro-Wilk Test: p-value = {p:.10f} (Residuals are not normally distributed)")

### Residuals are Normally distrubuted bot not centered around Zero (Indicating Bias) 

	•	The residuals show an overall normal distribution symmetry but are not centered around zero, it hints to bias in the model.
	•	This bias suggests that the model is systematically overestimating or underestimating 
	•	A better fit could potentially be achieved by adjusting the model 

## 1.3  - Comparing The Performance Of Different Regression Models

	Regression Models to Test:
	•	Linear Regression
	•	Ridge Regression
	•	Lasso Regression
	•	Decision Tree Regression
	•	Random Forest Regression
	Metrics for Comparison:
	•	 R^2  Score
	•	Mean Squared Error (MSE)
	•	Residual Analysis (to check the error distribution)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assuming `housing_data` is your dataset
X = housing_data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]
y = housing_data['MedHouseVal']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100)
}

# Results dictionary
results = {}

# Evaluate models
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    r2 = r2_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    residuals = y_test - predictions

    # Normality test for residuals
    shapiro_stat, shapiro_p = shapiro(residuals)
    normality = "Yes" if shapiro_p > 0.05 else "No"

    # Store results
    results[name] = {
        "Model": model,
        "R2": r2,
        "MSE": mse,
        "Residuals": residuals,
        "Shapiro-Wilk P-Value": shapiro_p,
        "Residuals Normally Distributed": normality
    }

# Visualize results
fig, axs = plt.subplots(len(models), 3, figsize=(18, 20))

for i, (name, result) in enumerate(results.items()):
    predictions = result["Model"].predict(X_test)
    residuals = result["Residuals"]

    # Predicted vs Actual
    axs[i, 0].scatter(y_test, predictions, alpha=0.6, label="Predicted vs Actual")
    axs[i, 0].plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle="--", label="Perfect Fit")
    axs[i, 0].set_title(f"{name} - Predicted vs Actual")
    axs[i, 0].set_xlabel("Actual")
    axs[i, 0].set_ylabel("Predicted")
    axs[i, 0].legend()

    # Residuals scatter plot
    axs[i, 1].scatter(predictions, residuals, alpha=0.6, label="Residuals")
    axs[i, 1].axhline(0, color="red", linestyle="--")
    axs[i, 1].set_title(f"{name} - Residuals")
    axs[i, 1].set_xlabel("Predicted")
    axs[i, 1].set_ylabel("Residuals")
    axs[i, 1].legend()

    # Residuals histogram and KDE
    sns.histplot(residuals, kde=True, ax=axs[i, 2], color="blue", bins=30, alpha=0.7, label="Residuals Distribution")
    axs[i, 2].set_title(f"{name} - Residuals Distribution")
    axs[i, 2].set_xlabel("Residuals")
    axs[i, 2].set_ylabel("Frequency")
    axs[i, 2].legend()

plt.tight_layout()
plt.show()

# Summary table
comparison_df = pd.DataFrame({
    "Model": [name for name in results.keys()],
    "R2 Score": [result["R2"] for result in results.values()],
    "MSE": [result["MSE"] for result in results.values()],
    "Shapiro-Wilk P-Value": [result["Shapiro-Wilk P-Value"] for result in results.values()],
    "Residuals Normally Distributed": [result["Residuals Normally Distributed"] for result in results.values()]
})

# Display results table
print("Model Performance Comparison:")
print(comparison_df)

# Optional visualization of final performance comparison
fig, ax = plt.subplots(figsize=(10, 6))
comparison_df.set_index("Model")[["R2 Score", "MSE"]].plot(kind="bar", ax=ax, alpha=0.7)
plt.title("Model Performance Comparison")
plt.ylabel("Score")
plt.tight_layout()
plt.show()

## 1.4 - Testing More ML Models And Comparing Performance. 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Assuming `housing_data` is your dataset
X = housing_data[['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']]
y = housing_data['MedHouseVal']

# Reduce dataset size for testing (optional)
# Uncomment the line below for faster testing
# X, y = X.sample(5000, random_state=42), y.sample(5000, random_state=42)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define models with optimized parameters
ml_models = {
   # "Support Vector Regression": SVR(kernel='linear', C=1.0),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42, n_estimators=50, max_depth=3),
    "XGBoost": XGBRegressor(random_state=42, n_estimators=50, max_depth=3, verbosity=0, n_jobs=-1),
    "K-Nearest Neighbors": KNeighborsRegressor(n_neighbors=5),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42, iterations=50, depth=3)
}

# Results dictionary
ml_results = {}

# Evaluate models
for name, model in ml_models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    r2 = r2_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    residuals = y_test - predictions

    # Normality test for residuals
    shapiro_stat, shapiro_p = shapiro(residuals)
    normality = "Yes" if shapiro_p > 0.05 else "No"

    # Store results
    ml_results[name] = {
        "Model": model,
        "R2": r2,
        "MSE": mse,
        "Residuals": residuals,
        "Shapiro-Wilk P-Value": shapiro_p,
        "Residuals Normally Distributed": normality
    }

# Visualize results
fig, axs = plt.subplots(len(ml_models), 3, figsize=(18, 20))

for i, (name, result) in enumerate(ml_results.items()):
    predictions = result["Model"].predict(X_test)
    residuals = result["Residuals"]

    # Predicted vs Actual
    axs[i, 0].scatter(y_test, predictions, alpha=0.6, label="Predicted vs Actual")
    axs[i, 0].plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle="--", label="Perfect Fit")
    axs[i, 0].set_title(f"{name} - Predicted vs Actual")
    axs[i, 0].set_xlabel("Actual")
    axs[i, 0].set_ylabel("Predicted")
    axs[i, 0].legend()

    # Residuals scatter plot
    axs[i, 1].scatter(predictions, residuals, alpha=0.6, label="Residuals")
    axs[i, 1].axhline(0, color="red", linestyle="--")
    axs[i, 1].set_title(f"{name} - Residuals")
    axs[i, 1].set_xlabel("Predicted")
    axs[i, 1].set_ylabel("Residuals")
    axs[i, 1].legend()

    # Residuals histogram and KDE
    sns.histplot(residuals, kde=True, ax=axs[i, 2], color="blue", bins=30, alpha=0.7, label="Residuals Distribution")
    axs[i, 2].set_title(f"{name} - Residuals Distribution")
    axs[i, 2].set_xlabel("Residuals")
    axs[i, 2].set_ylabel("Frequency")
    axs[i, 2].legend()
    

plt.tight_layout()
plt.show()

# Summary table
ml_comparison_df = pd.DataFrame({
    "Model": [name for name in ml_results.keys()],
    "R2 Score": [result["R2"] for result in ml_results.values()],
    "MSE": [result["MSE"] for result in ml_results.values()],
    "Shapiro-Wilk P-Value": [result["Shapiro-Wilk P-Value"] for result in ml_results.values()],
    "Residuals Normally Distributed": [result["Residuals Normally Distributed"] for result in ml_results.values()]
})

# Display results table
print("ML Model Performance Comparison:")
print(ml_comparison_df)